# Healthcare Payor Unity Catalog Tools
This notebook defines Unity Catalog functions as tools for the healthcare payor AI agent using existing tables in my_catalog.payer_silver.


In [1]:
# Set Catalog and Schema Parameters
CATALOG = "my_catalog"
SCHEMA = "payer_silver"

print(f"Working with catalog: {CATALOG}")
print(f"Working with schema: {SCHEMA}")
print(f"Creating UC functions in: {CATALOG}.{SCHEMA}")


Working with catalog: my_catalog
Working with schema: payer_silver
Creating UC functions in: my_catalog.payer_silver


### Create Member Lookup Function


In [2]:
# Create Member Lookup Function
spark.sql(f"DROP FUNCTION IF EXISTS {CATALOG}.{SCHEMA}.lookup_member")

sqlstr_lkp_member = f"""
CREATE OR REPLACE FUNCTION {CATALOG}.{SCHEMA}.lookup_member(
  input_id STRING COMMENT 'Input member id'
)
RETURNS TABLE (
    member_id STRING,
    first_name STRING,
    last_name STRING,
    birth_date DATE,
    gender STRING,
    plan_id STRING,
    effective_date DATE
)
COMMENT 'Returns member information from payer_silver.members'
RETURN (
  SELECT 
    member_id,
    first_name,
    last_name,
    birth_date,
    gender,
    plan_id,
    effective_date
  FROM {CATALOG}.{SCHEMA}.members
  WHERE member_id = input_id
);
"""

spark.sql(sqlstr_lkp_member)
print("✅ lookup_member function created successfully")


✅ lookup_member function created successfully


### Create Claims Lookup Function


In [4]:
# Create Claims Lookup Function
spark.sql(f"DROP FUNCTION IF EXISTS {CATALOG}.{SCHEMA}.lookup_claims")

sqlstr_lkp_claims = f"""
CREATE OR REPLACE FUNCTION {CATALOG}.{SCHEMA}.lookup_claims(
  input_id STRING COMMENT 'Input member id'
)
RETURNS TABLE (
    claim_id STRING,
    member_id STRING,
    provider_id STRING,
    claim_date DATE,
    total_charge DOUBLE,
    claim_status STRING
)
COMMENT 'Returns claims for a member from payer_silver.claims'
RETURN (
  SELECT 
    claim_id,
    member_id,
    provider_id,
    claim_date,
    total_charge,
    claim_status
  FROM {CATALOG}.{SCHEMA}.claims
  WHERE member_id = input_id
  ORDER BY claim_date DESC
);
"""

spark.sql(sqlstr_lkp_claims)
print("✅ lookup_claims function created successfully")


✅ lookup_claims function created successfully


### Create Provider Lookup Function


In [5]:
# Create Provider Lookup Function
spark.sql(f"DROP FUNCTION IF EXISTS {CATALOG}.{SCHEMA}.lookup_providers")

sqlstr_lkp_providers = f"""
CREATE OR REPLACE FUNCTION {CATALOG}.{SCHEMA}.lookup_providers(
  specialty_filter STRING COMMENT 'Specialty to filter providers'
)
RETURNS TABLE (
    provider_id STRING,
    provider_name STRING,
    specialty STRING,
    city STRING,
    state STRING
)
COMMENT 'Returns providers by specialty from payer_silver.providers'
RETURN (
  SELECT 
    provider_id,
    provider_name,
    specialty,
    city,
    state
  FROM {CATALOG}.{SCHEMA}.providers
  WHERE LOWER(specialty) LIKE LOWER(CONCAT('%', specialty_filter, '%'))
  ORDER BY provider_name
);
"""

spark.sql(sqlstr_lkp_providers)
print("✅ lookup_providers function created successfully")


✅ lookup_providers function created successfully


### Test All Functions


In [11]:
# Test Member Lookup Function
print("🧪 Testing lookup_member function...")
test_result = spark.sql(f"SELECT * FROM {CATALOG}.{SCHEMA}.lookup_member('1001') LIMIT 1")
test_result.show(truncate=False)


🧪 Testing lookup_member function...


+---------+----------+---------+----------+------+-------+--------------+
|member_id|first_name|last_name|birth_date|gender|plan_id|effective_date|
+---------+----------+---------+----------+------+-------+--------------+
|1001     |John      |Doe      |1980-03-21|M     |PLN101 |2020-01-01    |
+---------+----------+---------+----------+------+-------+--------------+



In [12]:
# Test Claims Lookup Function
print("🧪 Testing lookup_claims function...")
test_result = spark.sql(f"SELECT * FROM {CATALOG}.{SCHEMA}.lookup_claims('1001') LIMIT 3")
test_result.show(truncate=False)


🧪 Testing lookup_claims function...


+--------+---------+-----------+----------+------------+------------+
|claim_id|member_id|provider_id|claim_date|total_charge|claim_status|
+--------+---------+-----------+----------+------------+------------+
|CLM001  |1001     |2001       |2023-01-10|120.0       |paid        |
+--------+---------+-----------+----------+------------+------------+



In [13]:
# Test Provider Lookup Function
print("🧪 Testing lookup_providers function...")
test_result = spark.sql(f"SELECT * FROM {CATALOG}.{SCHEMA}.lookup_providers('Cardiology') LIMIT 3")
test_result.show(truncate=False)


🧪 Testing lookup_providers function...


+-----------+-------------+----------+----------+-----+
|provider_id|provider_name|specialty |city      |state|
+-----------+-------------+----------+----------+-----+
|2004       |Dr. Davis    |Cardiology|Louisville|KY   |
+-----------+-------------+----------+----------+-----+



### Summary


In [14]:
# Display Summary
print("🏥 Healthcare Payor Unity Catalog Tools Created Successfully!")
print("=" * 60)
print(f"✅ lookup_member - Member information from {CATALOG}.{SCHEMA}.members")
print(f"✅ lookup_claims - Member claims from {CATALOG}.{SCHEMA}.claims") 
print(f"✅ lookup_providers - Provider network from {CATALOG}.{SCHEMA}.providers")
print("=" * 60)
print("🎉 All healthcare payor tools are ready for agent integration!")
print(f"📋 Using existing tables from {CATALOG}.{SCHEMA}")


🏥 Healthcare Payor Unity Catalog Tools Created Successfully!
✅ lookup_member - Member information from my_catalog.payer_silver.members
✅ lookup_claims - Member claims from my_catalog.payer_silver.claims
✅ lookup_providers - Provider network from my_catalog.payer_silver.providers
🎉 All healthcare payor tools are ready for agent integration!
📋 Using existing tables from my_catalog.payer_silver
